In [10]:
# Imports
import polars as pl
import xgboost as xgb
import numpy as np
import optuna
import math
import statistics as stat

from lets_plot import *
from lets_plot.mapping import as_discrete
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import plotly
import plotly.figure_factory as ff

LetsPlot.setup_html()
plotly.offline.init_notebook_mode(connected = True)

In [11]:
df = pl.read_csv("traffic.csv", try_parse_dates = True).drop("ID")
df = df.filter(pl.col("Junction") == 1) # Take only the first junction
df = df.with_row_count(name = "Time_index", offset = 0) # add time index column
df.head()

Time_index,DateTime,Junction,Vehicles
u32,datetime[μs],i64,i64
0,2015-11-01 00:00:00,1,15
1,2015-11-01 01:00:00,1,13
2,2015-11-01 02:00:00,1,10
3,2015-11-01 03:00:00,1,7
4,2015-11-01 04:00:00,1,9


In [12]:
# Split into training and validation sets
df_train = df.filter(pl.col("DateTime") < pl.datetime(2017, 6, 1))
df_valid = df.filter(pl.col("DateTime") >= pl.datetime(2017, 6, 1))

In [13]:
# Initializing stuff
j1_color = "#EC7F6E"
linesize = 1

# Time series plot for each junction's training set
plt_ts = \
    ggplot(df_train)+\
    geom_line(aes(x = "DateTime", y = "Vehicles"), 
              color = j1_color, sampling = "none", size = linesize)+\
    scale_x_datetime(format = "%b %Y")+\
    theme_minimal2()+\
    theme(plot_title = element_text(hjust = 0.5, face = 'bold'))+\
    labs(x = "Date", y = "Vehicles", title = "Junction 1 Training Data")

ts_bunch = GGBunch()
ts_bunch.add_plot(plt_ts, 0, 0, 850, 300)
ts_bunch

In [14]:
# Pre-process data for the regression model
xtrain = df_train.get_column("Time_index").to_numpy()
xvalid = df_valid.get_column("Time_index").to_numpy()

xtrain = xtrain.reshape(-1,1)
xvalid = xvalid.reshape(-1,1)

ytrain = df_train.get_column("Vehicles").to_numpy()
yvalid = df_valid.get_column("Vehicles").to_numpy()

# Training 
trend_model = LinearRegression().fit(xtrain, ytrain)

# Predicting
trend_preds_valid = trend_model.predict(xvalid)

# Getting the RMSE
print("--------------------------------------------------------------")
print("Trend model validation set RMSE:", math.sqrt(mean_squared_error(yvalid, trend_preds_valid)))
print("--------------------------------------------------------------")

--------------------------------------------------------------
Trend model validation set RMSE: 27.24957995600769
--------------------------------------------------------------


In [15]:
# Initializing a color
true_values_color = "gray"
trend_model_color = '#C754F0'

# Getting the validation data for the plot
df_labels = pl.DataFrame(
    {'DateTime': df_valid.get_column("DateTime"), 
     'Vehicles': df_valid.get_column("Vehicles"), 
     'Group': ["Label"]*len(df_valid)}
)

df_preds = pl.DataFrame(
    {'DateTime_preds': df_valid.get_column("DateTime"), 
     'Vehicles_preds': trend_preds_valid, 
     'Group_preds': ["Predictions"]*len(df_valid)}
)

df_trend_results_valid = (
    pl.concat([df_labels, df_preds], how = 'horizontal')
    .with_columns(
        (pl.lit("True Values").alias("Group_label")),
        (pl.lit("Predictions").alias("Group_pred")))
)

# Plotting the predictions on the validation set
plt_reg_valid = \
    ggplot(df_trend_results_valid)+\
    geom_line(aes(x = "DateTime", y = "Vehicles", color = "Group_label"), 
              sampling = "none", size = linesize, show_legend = True)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_preds", color = "Group_pred"), 
              sampling = "none", size = linesize, show_legend = True)+\
    scale_color_manual(values = [true_values_color, trend_model_color])+\
    scale_x_datetime(format = "%Y-%m-%d")+\
    scale_y_continuous(limits = [20, 145])+\
    theme_minimal2()+\
    theme(plot_title = element_text(hjust = 0.5, face = 'bold'),
         legend_title = element_blank())+\
    labs(x = "Date", y = "Vehicles", title = "Linear Trend Model: Validation Set Predictions")

reg_bunch = GGBunch()
reg_bunch.add_plot(plt_reg_valid, 0, 0, 850, 300)
reg_bunch

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Pre-process data for LSTM model
xtrain_lstm = xtrain.reshape((xtrain.shape[0], 1, 1))
xvalid_lstm = xvalid.reshape((xvalid.shape[0], 1, 1))

# Define LSTM model architecture
model = tf.keras.Sequential([
    #layers.LSTM(50, activation='relu', input_shape=(1,xtrain.shape[1])),
    layers.Bidirectional(layers.LSTM(50, activation='relu'), input_shape=(1,xtrain.shape[1])),
   # layers.Conv1D(64, 2,activation='relu', kernel_size=3), 
    layers.Dense(64, activation='relu'),

    layers.Dropout(0.2),
    #layers.LSTM(50, activation='relu'),
    layers.Dense(64, activation = 'relu'),
    #layer.BatchNormalization(0.2),
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(0.2),
    layers.Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mse')

# Fit model
model.fit(xtrain_lstm, ytrain, epochs=50, batch_size=32, verbose=0)

# Predict on validation set
lstm_preds_valid__ = model.predict(xvalid_lstm)

# Getting the RMSE
print("--------------------------------------------------------------")
print("LSTM model validation set RMSE:", math.sqrt(mean_squared_error(yvalid, lstm_preds_valid)))
print("--------------------------------------------------------------")


23/23 [==============================] - 1s 3ms/step
--------------------------------------------------------------
LSTM model validation set RMSE: 67.9361158894584
--------------------------------------------------------------


In [17]:
# Initializing a color
true_values_color = "gray"
trend_model_color = '#C754F0'

# Getting the validation data for the plot
df_labels = pl.DataFrame(
    {'DateTime': df_valid.get_column("DateTime"), 
     'Vehicles': df_valid.get_column("Vehicles"), 
     'Group': ["Label"]*len(df_valid)}
)

df_preds = pl.DataFrame(
    {'DateTime_preds': df_valid.get_column("DateTime"), 
     'Vehicles_preds': lstm_preds_valid__, 
     'Group_preds': ["Predictions"]*len(df_valid)}
)

df_trend_results_valid = (
    pl.concat([df_labels, df_preds], how = 'horizontal')
    .with_columns(
        (pl.lit("True Values").alias("Group_label")),
        (pl.lit("Predictions").alias("Group_pred")))
)

# Plotting the predictions on the validation set
plt_reg_valid = \
    ggplot(df_trend_results_valid)+\
    geom_line(aes(x = "DateTime", y = "Vehicles", color = "Group_label"), 
              sampling = "none", size = linesize, show_legend = True)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_preds", color = "Group_pred"), 
              sampling = "none", size = linesize, show_legend = True)+\
    scale_color_manual(values = [true_values_color, trend_model_color])+\
    scale_x_datetime(format = "%Y-%m-%d")+\
    scale_y_continuous(limits = [20, 145])+\
    theme_minimal2()+\
    theme(plot_title = element_text(hjust = 0.5, face = 'bold'),
         legend_title = element_blank())+\
    labs(x = "Date", y = "Vehicles", title = "Linear Trend Model: Validation Set Predictions")

reg_bunch = GGBunch()
reg_bunch.add_plot(plt_reg_valid, 0, 0, 850, 300)
reg_bunch

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Pre-process data for LSTM model
xtrain_lstm = xtrain.reshape((xtrain.shape[0], 1, 1))
xvalid_lstm = xvalid.reshape((xvalid.shape[0], 1, 1))

# Define LSTM model architecture
model = tf.keras.Sequential([
    layers.LSTM(50, activation='relu', input_shape=(1, 1)),
    layers.Dense(64, activation='relu'),

    layers.Dropout(0.2),
    #layers.LSTM(50, activation='relu'),
    layers.Dense(64, activation = 'relu'),
    #layer.BatchNormalization(0.2),
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(0.2),
    layers.Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mse')

# Fit model
model.fit(xtrain_lstm, ytrain, epochs=50, batch_size=32, verbose=0)

# Predict on validation set
lstm_preds_valid = model.predict(xvalid_lstm)

# Getting the RMSE
print("--------------------------------------------------------------")
print("LSTM model validation set RMSE:", math.sqrt(mean_squared_error(yvalid, lstm_preds_valid)))
print("--------------------------------------------------------------")


23/23 [==============================] - 0s 3ms/step
--------------------------------------------------------------
LSTM model validation set RMSE: 31.126948704905722
--------------------------------------------------------------


In [20]:
# Features on the training set
df_train = df_train.with_columns(pl.col("DateTime").dt.year().alias("Year"))
df_train = df_train.with_columns(pl.col("DateTime").dt.month().alias("Month"))
df_train = df_train.with_columns(pl.col("DateTime").dt.day().alias("Day_month"))
df_train = df_train.with_columns(pl.col("DateTime").dt.weekday().alias("Day_week"))
df_train = df_train.with_columns(pl.col("DateTime").dt.hour().alias("Hour"))

# Features on the validation set
df_valid = df_valid.with_columns(pl.col("DateTime").dt.year().alias("Year"))
df_valid = df_valid.with_columns(pl.col("DateTime").dt.month().alias("Month"))
df_valid = df_valid.with_columns(pl.col("DateTime").dt.day().alias("Day_month"))
df_valid = df_valid.with_columns(pl.col("DateTime").dt.weekday().alias("Day_week"))
df_valid = df_valid.with_columns(pl.col("DateTime").dt.hour().alias("Hour"))

print(df_train.head())
print(df_valid.head())

shape: (5, 9)
┌────────────┬─────────────────────┬──────────┬──────────┬───┬───────┬───────────┬──────────┬──────┐
│ Time_index ┆ DateTime            ┆ Junction ┆ Vehicles ┆ … ┆ Month ┆ Day_month ┆ Day_week ┆ Hour │
│ ---        ┆ ---                 ┆ ---      ┆ ---      ┆   ┆ ---   ┆ ---       ┆ ---      ┆ ---  │
│ u32        ┆ datetime[μs]        ┆ i64      ┆ i64      ┆   ┆ u32   ┆ u32       ┆ u32      ┆ u32  │
╞════════════╪═════════════════════╪══════════╪══════════╪═══╪═══════╪═══════════╪══════════╪══════╡
│ 0          ┆ 2015-11-01 00:00:00 ┆ 1        ┆ 15       ┆ … ┆ 11    ┆ 1         ┆ 7        ┆ 0    │
│ 1          ┆ 2015-11-01 01:00:00 ┆ 1        ┆ 13       ┆ … ┆ 11    ┆ 1         ┆ 7        ┆ 1    │
│ 2          ┆ 2015-11-01 02:00:00 ┆ 1        ┆ 10       ┆ … ┆ 11    ┆ 1         ┆ 7        ┆ 2    │
│ 3          ┆ 2015-11-01 03:00:00 ┆ 1        ┆ 7        ┆ … ┆ 11    ┆ 1         ┆ 7        ┆ 3    │
│ 4          ┆ 2015-11-01 04:00:00 ┆ 1        ┆ 9        ┆ … ┆ 11    ┆ 1     

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df_train_ = scaler.fit_transform(df_train)
df_valid_ = scaler.transform(df_valid)

In [22]:
# Pre-process data for the regression model
xtrain = df_train.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()
xvalid = df_valid.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()

ytrain = df_train.get_column("Vehicles").to_numpy()
yvalid = df_valid.get_column("Vehicles").to_numpy()




from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

xtrain=scaler.fit_transform(xtrain)
xvalid=scaler.transform(xvalid)

# ytrain= scaler.fit_transform(ytrain)
# yvalid= scaler.transform(yvalid)



# Training 
reg_model = LinearRegression().fit(xtrain, ytrain)

# Predicting
reg_preds_valid = reg_model.predict(xvalid)

# Getting the RMSE
print("---------------------------------------------------------------------------")
print("Linear model + more features validation set RMSE:", 
      math.sqrt(mean_squared_error(yvalid, reg_preds_valid)))
print("----------------------------------------------------------------------------")

---------------------------------------------------------------------------
Linear model + more features validation set RMSE: 21.36780895658643
----------------------------------------------------------------------------


In [23]:
# Initializing a color
reg_model_color = '#1CA699'

# Getting the validation data for the plot
df_preds = pl.DataFrame(
    {'DateTime_preds': df_valid.get_column("DateTime"), 
     'Vehicles_preds': reg_preds_valid, 
     'Group_preds': ["Predictions"]*len(df_valid)}
)

df_reg_results_valid = (
    pl.concat([df_labels, df_preds], how = 'horizontal')
    .with_columns(
        (pl.lit("True Values").alias("Group_label")),
        (pl.lit("Predictions").alias("Group_pred")))
)

# Plotting the predictions on the validation set
plt_reg_valid = \
    ggplot(df_reg_results_valid)+\
    geom_line(aes(x = "DateTime", y = "Vehicles", color = "Group_label"), 
              sampling = "none", size = linesize, show_legend = True)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_preds", color = "Group_pred"), 
              sampling = "none", size = linesize, show_legend = True)+\
    scale_color_manual(values = [true_values_color, reg_model_color])+\
    scale_x_datetime(format = "%Y-%m-%d")+\
    scale_y_continuous(limits = [20, 145])+\
    theme_minimal2()+\
    theme(plot_title = element_text(hjust = 0.5, face = 'bold'),
         legend_title = element_blank())+\
    labs(x = "Date", y = "Vehicles", title = "Linear Model: Validation Set Predictions")

reg_bunch = GGBunch()
reg_bunch.add_plot(plt_reg_valid, 0, 0, 850, 300)
reg_bunch

In [26]:

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error
import math

# Pre-process data for LSTM model
xtrain_lstm = xtrain.reshape((xtrain.shape[0], 1, xtrain.shape[1]))
xvalid_lstm = xvalid.reshape((xvalid.shape[0], 1, xvalid.shape[1]))

# Define LSTM model architecture
model = tf.keras.Sequential([
    #layers.LSTM(50, activation='relu', input_shape=(1,xtrain.shape[1])),
    layers.Bidirectional(layers.LSTM(50, activation='relu'), input_shape=(1,xtrain.shape[1])),
   # layers.Conv1D(64, 2,activation='relu', kernel_size=3), 
    layers.Dense(64, activation='relu'),

    layers.Dropout(0.2),
    #layers.LSTM(50, activation='relu'),
    layers.Dense(64, activation = 'relu'),
    #layer.BatchNormalization(0.2),
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(0.2),
    layers.Dense(1)
])
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='mse')

# Fit model
model.fit(xtrain_lstm, ytrain, epochs=40, batch_size=32, verbose=0)

# Predict on validation set
lstm_preds_valid = model.predict(xvalid_lstm)

# Getting the RMSE
print("--------------------------------------------------------------")
print("LSTM model validation set RMSE:", math.sqrt(mean_squared_error(yvalid, lstm_preds_valid)))
print("--------------------------------------------------------------")


KeyboardInterrupt: 

In [27]:
# Suppress optuna log messages
optuna.logging.set_verbosity(optuna.logging.WARNING) 

# Optuna objective function
def objective_xgb(trial):
    """
    Optuna objective function. Returns
    the RMSE for an XGBoost model
    
    Assumes the training data are 
    polars data frames
    """
    # Get data for the XGBoost model
    xtrain = df_train.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()
    xvalid = df_valid.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()

    ytrain = df_train.get_column("Vehicles").to_numpy()
    yvalid = df_valid.get_column("Vehicles").to_numpy()
    
    dmat_train = xgb.DMatrix(xtrain, label = ytrain)
    dmat_valid = xgb.DMatrix(xvalid, label = yvalid)
    
    # Suggest hyperparameters for XGBoost
    params = {'objective': 'reg:squarederror',
              'eval_metric': 'rmse',
              'seed': 19970507,
              'eta': trial.suggest_float("eta", 1e-2, 0.25, log = True),
              'max_depth': trial.suggest_int("max_depth", 1, 14),
              'lambda': trial.suggest_float("lambda", 1e-8, 100.0, log = True),
              'alpha': trial.suggest_float("alpha", 1e-8, 100.0, log = True), 'learning_rate':0.02,
             }
    
    # To evaluate training progress (set verbose_eval = True)
    watchlist = [(dmat_train, 'train'), (dmat_valid, 'eval')]
    
    # Train the XGBoost model
    xgb_model = xgb.train(params, 
                          dtrain = dmat_train, 
                          num_boost_round = trial.suggest_int("num_boost_round", 20, 3000),
                          evals = watchlist,
                          verbose_eval = False)
    
    xgb_preds_valid = xgb_model.predict(dmat_valid) 
    
    # Return the RMSE
    return math.sqrt(mean_squared_error(yvalid, xgb_preds_valid))


# Set up and run the Optuna study
study_xgb = optuna.create_study(direction = 'minimize')
study_xgb.optimize(objective_xgb, n_trials = 10)

# Create a table showing the best parameters
xgb_table = [["Parameter", "Optimal Value from Optuna"],
            ["Iterations (num_boost_rounds)", study_xgb.best_params['num_boost_round']],
            ['Learning Rate (eta)', round(study_xgb.best_params['eta'], 3)],
            ['Max Depth (max_depth)', round(study_xgb.best_params['max_depth'], 3)],
            ['Lambda (lambda)', round(study_xgb.best_params['lambda'], 3)],
            ['Alpha (alpha)', round(study_xgb.best_params['alpha'], 3)]]

ff.create_table(xgb_table)

In [28]:
# Taking the model with the best hyperparameters and testing it
xtrain = df_train.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()
xvalid = df_valid.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()



ytrain = df_train.get_column("Vehicles").to_numpy()
yvalid = df_valid.get_column("Vehicles").to_numpy()


    
dmat_train = xgb.DMatrix(xtrain, label = ytrain)
dmat_valid = xgb.DMatrix(xvalid, label = yvalid)

best_params = {'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'seed': 19970507,
                'eta': study_xgb.best_params['eta'],
                'max_depth': study_xgb.best_params['max_depth'],
                'lambda': study_xgb.best_params['lambda'],
                'alpha': study_xgb.best_params['alpha'],
                 }

xgb_model = xgb.train(best_params, 
                      dtrain = dmat_train, 
                      num_boost_round = study_xgb.best_params['num_boost_round'],
                      verbose_eval = False)

xgb_preds_valid = xgb_model.predict(dmat_valid)

print('----------------------------------------------------------')
print('XGBoost validation set RMSE:', math.sqrt(mean_squared_error(yvalid, xgb_preds_valid)))
print('----------------------------------------------------------')

----------------------------------------------------------
XGBoost validation set RMSE: 7.042297696627067
----------------------------------------------------------


In [29]:
# Initializing a color
xgb_color = '#F78B38'

# Getting the validation data for the plot
df_preds = pl.DataFrame(
    {'DateTime_preds': df_valid.get_column("DateTime"), 
     'Vehicles_preds': xgb_preds_valid, 
     'Group_preds': ["Predictions"]*len(df_valid)}
)

df_xgb_results_valid = (
    pl.concat([df_labels, df_preds], how = 'horizontal')
    .with_columns(
        (pl.lit("True Values").alias("Group_label")),
        (pl.lit("Predictions").alias("Group_pred")))
)

# Plotting the predictions on the validation set
plt_xgb_valid = \
    ggplot(df_xgb_results_valid)+\
    geom_line(aes(x = "DateTime", y = "Vehicles", color = "Group_label"), 
              sampling = "none", size = linesize, show_legend = True)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_preds", color = "Group_pred"), 
              sampling = "none", size = linesize, show_legend = True)+\
    scale_color_manual(values = [true_values_color, xgb_color])+\
    scale_x_datetime(format = "%Y-%m-%d")+\
    scale_y_continuous(limits = [20, 145])+\
    theme_minimal2()+\
    theme(plot_title = element_text(hjust = 0.5, face = 'bold'),
         legend_title = element_blank())+\
    labs(x = "Date", y = "Vehicles", title = "XGBoost: Validation Set Predictions")

xgb_bunch = GGBunch()
xgb_bunch.add_plot(plt_xgb_valid, 0, 0, 850, 300)
xgb_bunch

In [28]:
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import math

# Suppress TensorFlow log messages
import tensorflow as tf
tf.get_logger().setLevel('ERROR')


# Define the LSTM model
def create_lstm_model(trial, input_shape):
    model = Sequential()

    # Add the first LSTM layer and Dropout regularization
    model.add(LSTM(units = trial.suggest_int('lstm_units', 16, 256, log=True), 
                   input_shape = input_shape, 
                   return_sequences = True))
    model.add(Dropout(rate = trial.suggest_float('dropout_1', 0.0, 0.5)))

    # Add the second LSTM layer and Dropout regularization
    model.add(LSTM(units = trial.suggest_int('lstm_units', 16, 256, log=True), 
                   return_sequences = False))
    model.add(Dropout(rate = trial.suggest_float('dropout_2', 0.0, 0.5)))

    # Add the output layer
    model.add(Dense(units = 1))

    return model


# Define the objective function to minimize (RMSE)
def objective_lstm(trial):
    # Get data for the LSTM model
    xtrain = df_train.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()
    xvalid = df_valid.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()

    xtrain=scaler.fit_transform(xtrain)
    xvalid=scaler.transform(xvalid)
    
    ytrain = df_train.get_column("Vehicles").to_numpy()
    yvalid = df_valid.get_column("Vehicles").to_numpy()

    input_shape = (xtrain.shape[1], 1)

    # Create the LSTM model with the suggested hyperparameters
    lstm_model = create_lstm_model(trial, input_shape)
    lstm_model.compile(loss = 'mean_squared_error', optimizer = 'adam')

    # Set up early stopping to avoid overfitting
    es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 0, patience = 10)

    # Train the LSTM model
    lstm_history = lstm_model.fit(xtrain.reshape(-1, xtrain.shape[1], 1), 
                                   ytrain, 
                                   epochs = 300, 
                                   batch_size = 32,
                                   validation_data = (xvalid.reshape(-1, xvalid.shape[1], 1), yvalid),
                                   callbacks = [es],
                                   verbose = 0)

    lstm_preds_valid = lstm_model.predict(xvalid.reshape(-1, xvalid.shape[1], 1))

    # Return the RMSE
    return math.sqrt(mean_squared_error(yvalid, lstm_preds_valid))


# Set up and run the Optuna study
study_lstm = optuna.create_study(direction = 'minimize')
study_lstm.optimize(objective_lstm, n_trials = 10)

# Create a table showing the best parameters
lstm_table = [["Parameter", "Optimal Value from Optuna"],
            ["LSTM Units", study_lstm.best_params['lstm_units']],
            ['Dropout Rate 1', round(study_lstm.best_params['dropout_1'], 3)],
            ['Dropout Rate 2', round(study_lstm.best_params['dropout_2'], 3)]]

ff.create_table(lstm_table)


23/23 [==============================] - 2s 16ms/step


In [29]:
# Define the function to create and train the best LSTM model
def train_best_lstm():
    # Get data for the LSTM model
    xtrain = df_train.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()
    xtest = df_valid.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()

    ytrain = df_train.get_column("Vehicles").to_numpy()
    ytest = df_valid.get_column("Vehicles").to_numpy()

    input_shape = (xtrain.shape[1], 1)

    # Create the LSTM model with the best hyperparameters found by Optuna
    lstm_model = Sequential()
    lstm_model.add(LSTM(units=study_lstm.best_params['lstm_units'], 
                        input_shape=input_shape, 
                        return_sequences=True))
    lstm_model.add(Dropout(rate=study_lstm.best_params['dropout_1']))
    lstm_model.add(LSTM(units=study_lstm.best_params['lstm_units'], 
                        return_sequences=False))
    lstm_model.add(Dropout(rate=study_lstm.best_params['dropout_2']))
    lstm_model.add(Dense(units=1))
    lstm_model.compile(loss='mean_squared_error', optimizer='adam')

    # Train the LSTM model on the full training set
    lstm_model.fit(xtrain.reshape(-1, xtrain.shape[1], 1), 
                   ytrain, 
                   epochs=100, 
                   batch_size=32,
                   verbose=0)

    # Make predictions on the test set
    lstm_preds_test = lstm_model.predict(xtest.reshape(-1, xtest.shape[1], 1))

    # Calculate the RMSE of the model's predictions on the test set
    lstm_rmse = math.sqrt(mean_squared_error(ytest, lstm_preds_test))

    return lstm_rmse

# Call the function to train the best LSTM model and get the error rate
lstm_error_rate = train_best_lstm()
print(f"LSTM error rate on test data: {lstm_error_rate:.3f}")


23/23 [==============================] - 1s 5ms/step
LSTM error rate on test data: 38.335


In [30]:
# Suppress optuna log messages
optuna.logging.set_verbosity(optuna.logging.WARNING) 

# Optuna objective function
def objective_trend_xgb(trial):
    
    # Get data for the trend model
    xtrain_reg = df_train.get_column("Time_index").to_numpy()
    xvalid_reg = df_valid.get_column("Time_index").to_numpy()

    xtrain_reg = xtrain_reg.reshape(-1,1)
    xvalid_reg = xvalid_reg.reshape(-1,1)
    
    ytrain = df_train.get_column("Vehicles").to_numpy()
    yvalid = df_valid.get_column("Vehicles").to_numpy()
    
    # Train and predict w/ the trend model
    reg_model = LinearRegression().fit(xtrain_reg, ytrain)

    # Predicting
    reg_preds_train = reg_model.predict(xtrain_reg)
    reg_preds_valid = reg_model.predict(xvalid_reg)
    
    # Calculate the residuals
    reg_resids_train = (ytrain - reg_preds_train)
    reg_resids_valid = (yvalid - reg_preds_valid)
    
    # Get the data for the XGB model
    xtrain_xgb = df_train.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()
    xvalid_xgb = df_valid.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()
    
    dmat_train = xgb.DMatrix(xtrain_xgb, label = reg_resids_train)
    dmat_valid = xgb.DMatrix(xvalid_xgb, label = reg_resids_valid)
    
    # Suggest hyperparameters
    params = {'objective': 'reg:squarederror',
              'eval_metric': 'rmse',
              'seed': 19970507,
              'eta': trial.suggest_float("eta", 1e-2, 0.25, log = True),
              'max_depth': trial.suggest_int("max_depth", 1, 14),
              'lambda': trial.suggest_float("lambda", 1e-8, 100.0, log = True),
              'alpha': trial.suggest_float("alpha", 1e-8, 100.0, log = True),
               'learning_rate':0.02,
             }
    
    # To evaluate training progress (set verbose_eval = True)
    watchlist = [(dmat_train, 'train'), (dmat_valid, 'eval')]
    
    # Train and predict w/ the XGBoost model
    xgb_model = xgb.train(params, 
                          dtrain = dmat_train, 
                          num_boost_round = trial.suggest_int("num_boost_round", 20, 3000),
                          evals = watchlist,
                          verbose_eval = False)
    
    xgb_preds_valid = xgb_model.predict(dmat_valid) 
    
    # Sum the final predictions
    trend_xgb_preds_valid = (reg_preds_valid + xgb_preds_valid)
    
    # Return the RMSE
    return math.sqrt(mean_squared_error(yvalid,  trend_xgb_preds_valid))
    

# Set up and run the Optuna study
study_trend_xgb = optuna.create_study(direction = 'minimize')
study_trend_xgb.optimize(objective_trend_xgb, n_trials = 10)

# Create a table showing the best parameters
trend_xgb_table = [["Parameter", "Optimal Value from Optuna"],
                  ["Iterations (num_boost_rounds)", study_trend_xgb.best_params['num_boost_round']],
                  ['Learning Rate (eta)', round(study_trend_xgb.best_params['eta'], 3)],
                  ['Max Depth (max_depth)', round(study_trend_xgb.best_params['max_depth'], 3)],
                  ['Lambda', round(study_trend_xgb.best_params['lambda'], 3)],
                  ['Alpha', round(study_trend_xgb.best_params['alpha'], 3)]]

ff.create_table(trend_xgb_table)

In [31]:
# Taking the model with the best hyperparameters and testing it

# Get data for the trend model
xtrain_reg = df_train.get_column("Time_index").to_numpy()
xvalid_reg = df_valid.get_column("Time_index").to_numpy()

xtrain_reg = xtrain_reg.reshape(-1,1)
xvalid_reg = xvalid_reg.reshape(-1,1)
    
ytrain = df_train.get_column("Vehicles").to_numpy()
yvalid = df_valid.get_column("Vehicles").to_numpy()
    
# Train and predict w/ the trend model
reg_model = LinearRegression().fit(xtrain_reg, ytrain)

# Predicting
reg_preds_train = reg_model.predict(xtrain_reg)
reg_preds_valid = reg_model.predict(xvalid_reg)
    
# Calculate the residuals
reg_resids_train = (ytrain - reg_preds_train)
reg_resids_valid = (yvalid - reg_preds_valid)
    
# Get the data for the XGB model
xtrain_xgb = df_train.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()
xvalid_xgb = df_valid.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()
    
dmat_train = xgb.DMatrix(xtrain_xgb, label = reg_resids_train)
dmat_valid = xgb.DMatrix(xvalid_xgb, label = reg_resids_valid)

best_params = {'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'seed': 19970507,
                'eta': study_trend_xgb.best_params['eta'],
                'max_depth': study_trend_xgb.best_params['max_depth'],
                'lambda': study_trend_xgb.best_params['lambda'],
                'alpha': study_trend_xgb.best_params['alpha'],
                 }

xgb_model = xgb.train(best_params, 
                      dtrain = dmat_train, 
                      num_boost_round = study_trend_xgb.best_params['num_boost_round'],
                      verbose_eval = False)

xgb_preds_valid = xgb_model.predict(dmat_valid)

# Sum the final predictions
trend_xgb_preds_valid = (reg_preds_valid + xgb_preds_valid)

print('----------------------------------------------------------')
print('Trend model + XGBoost validation set RMSE:', math.sqrt(mean_squared_error(yvalid, trend_xgb_preds_valid)))
print('----------------------------------------------------------')

----------------------------------------------------------
Trend model + XGBoost validation set RMSE: 7.008927893854633
----------------------------------------------------------


In [32]:
# Initializing a color
trend_xgb_color = '#F73838'

# Getting the validation data for the plot
df_preds = pl.DataFrame(
    {'DateTime_preds': df_valid.get_column("DateTime"), 
     'Vehicles_preds': trend_xgb_preds_valid, 
     'Group_preds': ["Predictions"]*len(df_valid)}
)

df_trend_xgb_results_valid = (
    pl.concat([df_labels, df_preds], how = 'horizontal')
    .with_columns(
        (pl.lit("True Values").alias("Group_label")),
        (pl.lit("Predictions").alias("Group_pred")))
)

# Plotting the predictions on the validation set
plt_trend_xgb_valid = \
    ggplot(df_trend_xgb_results_valid)+\
    geom_line(aes(x = "DateTime", y = "Vehicles", color = "Group_label"), 
              sampling = "none", size = linesize, show_legend = True)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_preds", color = "Group_pred"), 
              sampling = "none", size = linesize, show_legend = True)+\
    scale_color_manual(values = [true_values_color, trend_xgb_color])+\
    scale_x_datetime(format = "%Y-%m-%d")+\
    scale_y_continuous(limits = [20, 145])+\
    theme_minimal2()+\
    theme(plot_title = element_text(hjust = 0.5, face = 'bold'),
         legend_title = element_blank())+\
    labs(x = "Date", y = "Vehicles", title = "Trend Model + XGBoost: Validation Set Predictions")

trend_xgb_bunch = GGBunch()
trend_xgb_bunch.add_plot(plt_trend_xgb_valid, 0, 0, 850, 300)
trend_xgb_bunch

In [33]:
# Lagged Features
print(pl.concat([df_train.select(pl.col("Vehicles")), 
                 df_train.select([pl.col("Vehicles").shift(1).alias("Lag 1")]),
                 df_train.select([pl.col("Vehicles").shift(2).alias("Lag 2")]),
                 df_train.select([pl.col("Vehicles").shift(3).alias("Lag 3")]),
                 df_train.select([pl.col("Vehicles").shift(4).alias("Lag 4")])], how = 'horizontal').head())

# Adding two lagged features to the model
one_month = 24*30 # To match validation set length month = 30 days
one_year = 24*364

df_train = df_train.with_columns([
    (pl.col("Vehicles").shift(one_month).alias("Lag_month")),
    (pl.col("Vehicles").shift(one_year).alias("Lag_year"))
])

shape: (5, 5)
┌──────────┬───────┬───────┬───────┬───────┐
│ Vehicles ┆ Lag 1 ┆ Lag 2 ┆ Lag 3 ┆ Lag 4 │
│ ---      ┆ ---   ┆ ---   ┆ ---   ┆ ---   │
│ i64      ┆ i64   ┆ i64   ┆ i64   ┆ i64   │
╞══════════╪═══════╪═══════╪═══════╪═══════╡
│ 15       ┆ null  ┆ null  ┆ null  ┆ null  │
│ 13       ┆ 15    ┆ null  ┆ null  ┆ null  │
│ 10       ┆ 13    ┆ 15    ┆ null  ┆ null  │
│ 7        ┆ 10    ┆ 13    ┆ 15    ┆ null  │
│ 9        ┆ 7     ┆ 10    ┆ 13    ┆ 15    │
└──────────┴───────┴───────┴───────┴───────┘


In [34]:
# Now the validaiton set needs lags
# Note the validation set length is one month
df_valid = df_valid.with_columns([
    (df_train.get_column("Vehicles")[-one_month:].alias("Lag_month")),
    (df_train.get_column("Vehicles")[-one_year:(-one_year + one_month)].alias("Lag_year"))
])

# Set up and run the Optuna study
study_xgb_lag = optuna.create_study(direction = 'minimize')
study_xgb_lag.optimize(objective_xgb, n_trials = 10)

# Create a table showing the best parameters
xgb_lag_table = [["Parameter", "Optimal Value from Optuna"],
                 ["Iterations (num_boost_rounds)", study_xgb_lag.best_params['num_boost_round']],
                 ['Learning Rate (eta)', round(study_xgb_lag.best_params['eta'], 3)],
                 ['Max Depth (max_depth)', round(study_xgb_lag.best_params['max_depth'], 3)],
                 ['Lambda (lambda)', round(study_xgb_lag.best_params['lambda'], 3)],
                 ['Alpha (alpha)', round(study_xgb_lag.best_params['alpha'], 3)]]

ff.create_table(xgb_lag_table)

In [35]:
# Taking the model with the best hyperparameters and testing it
xtrain = df_train.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()
xvalid = df_valid.drop(["DateTime", "Junction", "Vehicles"]).to_numpy()

ytrain = df_train.get_column("Vehicles").to_numpy()
yvalid = df_valid.get_column("Vehicles").to_numpy()
    
dmat_train = xgb.DMatrix(xtrain, label = ytrain)
dmat_valid = xgb.DMatrix(xvalid, label = yvalid)

best_params = {'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'seed': 19970507,
                'eta': study_xgb_lag.best_params['eta'],
                'max_depth': study_xgb_lag.best_params['max_depth'],
                'lambda': study_xgb_lag.best_params['lambda'],
                'alpha': study_xgb_lag.best_params['alpha'],
                 }

xgb_lag_model = xgb.train(best_params, 
                         dtrain = dmat_train, 
                         num_boost_round = study_xgb_lag.best_params['num_boost_round'],
                         verbose_eval = False)

xgb_lag_preds_valid = xgb_lag_model.predict(dmat_valid)

print('----------------------------------------------------------')
print('XGBoost (w/ Lagging) validation set RMSE:', math.sqrt(mean_squared_error(yvalid, xgb_lag_preds_valid)))
print('----------------------------------------------------------')

----------------------------------------------------------
XGBoost (w/ Lagging) validation set RMSE: 8.19653288648766
----------------------------------------------------------


In [36]:
# Initializing a color
xgb_lag_color = '#F7F738'

# Getting the validation data for the plot
df_preds = pl.DataFrame(
    {'DateTime_preds': df_valid.get_column("DateTime"), 
     'Vehicles_preds': xgb_lag_preds_valid, 
     'Group_preds': ["Predictions"]*len(df_valid)}
)

df_xgb_lag_results_valid = (
    pl.concat([df_labels, df_preds], how = 'horizontal')
    .with_columns(
        (pl.lit("True Values").alias("Group_label")),
        (pl.lit("Predictions").alias("Group_pred")))
)

# Plotting the predictions on the validation set
plt_xgb_lag_valid = \
    ggplot(df_xgb_lag_results_valid)+\
    geom_line(aes(x = "DateTime", y = "Vehicles", color = "Group_label"), 
              sampling = "none", size = linesize, show_legend = True)+\
    geom_line(aes(x = "DateTime", y = "Vehicles_preds", color = "Group_pred"), 
              sampling = "none", size = linesize, show_legend = True)+\
    scale_color_manual(values = [true_values_color, xgb_color])+\
    scale_x_datetime(format = "%Y-%m-%d")+\
    scale_y_continuous(limits = [20, 145])+\
    theme_minimal2()+\
    theme(plot_title = element_text(hjust = 0.5, face = 'bold'),
         legend_title = element_blank())+\
    labs(x = "Date", y = "Vehicles", title = "XGBoost (w/ Lagging): Validation Set Predictions")

xgb_lag_bunch = GGBunch()
xgb_lag_bunch.add_plot(plt_xgb_lag_valid, 0, 0, 850, 300)
xgb_lag_bunch

